In [1]:
#Import
import pandas as pd
import numpy as np

import json
from pandas.io.json import json_normalize

In [2]:
# Visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_json('../Data/V10/dblp-ref/dblp-ref-0.json', lines=True)

In [4]:
from  itertools import chain

df_flatten_authors = pd.DataFrame({
        "abstract": np.repeat(df.abstract.values, df.authors.str.len()),
        "authors": list(chain.from_iterable(df.authors)),
        "id": np.repeat(df.id.values, df.authors.str.len()),
        "n_citation": np.repeat(df.n_citation.values, df.authors.str.len()),
        "references": np.repeat(df.references.values, df.authors.str.len()),
        "title": np.repeat(df.title.values, df.authors.str.len()),
        "venue": np.repeat(df.venue.values, df.authors.str.len()),
        "year": np.repeat(df.year.values, df.authors.str.len())
})

In [5]:
df_copy = df.copy()
df_flatten_authors_copy = df_flatten_authors.copy()

In [6]:
def h_index_utility(series):
    count = 0
    sortedSeries = np.sort(series)
    sortedSeries = sortedSeries[::-1]
    i = 1
    for item in sortedSeries:
        if (item < i): #check on equality
            break
        i+=1    
    #series['h_index'] = (i-1)
    return (i-1)

In [7]:
df_flatten_authors_h_index = df_flatten_authors.groupby('authors').agg({'n_citation' : ['sum', h_index_utility]})
df_flatten_authors_h_index.columns = ["_".join(x) for x in df_flatten_authors_h_index.columns.ravel()]
df_flatten_authors_h_index['author_name'] = df_flatten_authors_h_index.index

#df_flatten_authors_h_index.sort_values(by=['n_citation_h_index_utility'], ascending=False).head(5)
#df_flatten_authors_h_index.sort_values(by=['n_citation_sum'], ascending=False).head(5)

#graph made in project proposal
"""
g = sns.jointplot(x="n_citation_sum", y="n_citation_h_index_utility", data=df_flatten_authors_h_index)
g.set_axis_labels('Citation Count', 'h-index', fontsize=10, fontweight='bold')
g.savefig('h_index_n_citation_count_Nov_11.png')
"""

#df_flatten_authors[df_flatten_authors['authors'] == 'David E. Goldberg'].sort_values(by=['n_citation'], ascending=False)


'\ng = sns.jointplot(x="n_citation_sum", y="n_citation_h_index_utility", data=df_flatten_authors_h_index)\ng.set_axis_labels(\'Citation Count\', \'h-index\', fontsize=10, fontweight=\'bold\')\ng.savefig(\'h_index_n_citation_count_Nov_11.png\')\n'

In [12]:
df_flatten_authors_h_index.dtypes

n_citation_sum                 int64
n_citation_h_index_utility     int64
author_name                   object
dtype: object

In [8]:
#tried to rank on h-index

"""
df_flatten_authors_copy3_2['rank'] = df_flatten_authors_copy3_2['n_citation_h_index_utility2'].rank(ascending=False)
df_flatten_authors_copy3_2.head()
sns.jointplot(x="rank", y="n_citation_sum", data=df_flatten_authors_copy3_2)

df_flatten_authors_copy3_3 = df_flatten_authors_copy3_2.sample(n=100000)
sns.jointplot(x="n_citation_sum", y="n_citation_h_index_utility2", data=df_flatten_authors_copy3_3)

"""

'\ndf_flatten_authors_copy3_2[\'rank\'] = df_flatten_authors_copy3_2[\'n_citation_h_index_utility2\'].rank(ascending=False)\ndf_flatten_authors_copy3_2.head()\nsns.jointplot(x="rank", y="n_citation_sum", data=df_flatten_authors_copy3_2)\n\ndf_flatten_authors_copy3_3 = df_flatten_authors_copy3_2.sample(n=100000)\nsns.jointplot(x="n_citation_sum", y="n_citation_h_index_utility2", data=df_flatten_authors_copy3_3)\n\n'

In [13]:
""" IMPORTANCE Calculation"""

' IMPORTANCE Calculation'

In [14]:
#inserting empty array for nan

df.loc[df['references'].isnull(),['references']] = df.loc[df['references'].isnull(),'references'].apply(lambda references : [])

#placing [self] where references is nan

df.loc[df['references'].isnull(),['references']] = df.loc[df['references'].isnull(),'references'].apply(lambda references : ['self'])

#without [self], empty array, on flatten entries with empty array will be deleted

"""
df = df_copy.copy()
df.head()
"""

'\ndf = df_copy.copy()\ndf.head()\n'

In [15]:
from  itertools import chain

df_flatten_references = pd.DataFrame({
        "abstract": np.repeat(df.abstract.values, df.references.str.len()),
        "authors": np.repeat(df.authors.values, df.references.str.len()),
        "id": np.repeat(df.id.values, df.references.str.len()),
        "n_citation": np.repeat(df.n_citation.values, df.references.str.len()),
        "references": list(chain.from_iterable(df.references)),
        "title": np.repeat(df.title.values, df.references.str.len()),
        "venue": np.repeat(df.venue.values, df.references.str.len()),
        "year": np.repeat(df.year.values, df.references.str.len())
})

In [16]:
#df_flatten_references.loc[df_flatten_references['id'] == '00745041-3636-4d18-bbec-783c4278c40d']
#df_flatten_references[df_flatten_references['id'] == '00745041-3636-4d18-bbec-783c4278c40d']
#df_flatten_references.loc[df_flatten_references['references'] == '51c7e02e-f5ed-431a-8cf5-f761f266d4be']

In [17]:
df_flatten_references_importance = df_flatten_references.groupby('references').agg({'n_citation' : 'sum'})

In [18]:
#mistake 

#df['references'] = df.index

#df_flatten_references['references'] = df_flatten_references.index

df_flatten_references_importance['id'] = df_flatten_references_importance.index

df_flatten_references_importance['importance'] = df_flatten_references_importance['n_citation']

In [19]:
#df_flatten_references_importance.dtypes
#df_flatten_references_importance[df_flatten_references_importance['id'] == '51c7e02e-f5ed-431a-8cf5-f761f266d4be']


In [20]:
""" MERGE """

' MERGE '

In [21]:
df_result = pd.merge(df, df_flatten_references_importance[['id', 'importance']], on='id')

In [22]:
#df_result.loc[df_result['id'] == '51c7e02e-f5ed-431a-8cf5-f761f266d4be']
#df_result.loc[df_result['id'] == '00001301-0e56-4c9d-94aa-f776580aed87']
#df_result.shape

""" Pickel : Save and Read """

"""
df_result.to_pickle('pickle1')
df_result2 = pd.read_pickle('pickle1')
df_result2.shape()
"""


"\ndf_result.to_pickle('pickle1')\ndf_result2 = pd.read_pickle('pickle1')\ndf_result2.shape()\n"